# Broad Line Fitting with PyRSRMC (New MCMC)

This notebook demonstrates the new, clean MCMC fitter rebuilt from scratch.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PyRSRMC import run_mcmc_fit

%matplotlib inline

## 1. Load Data
Here we create a mock spectrum, but you would load your `.fits` file.

In [ ]:
# Mock Data Generation
z = 6.1
lam = np.linspace(3.0, 5.0, 1000) # microns

# Continuum
flux = np.full_like(lam, 0.1)

# Add Line (H-alpha)
lam_ha = 0.6563 * (1+z)
sigma_um = 0.005
flux += 5.0 * np.exp(-0.5*((lam-lam_ha)/sigma_um)**2)

# Noise
err = np.full_like(lam, 0.2)
flux += np.random.normal(0, 0.2, len(lam))

spec_data = {"lam": lam, "flux": flux, "err": err}

plt.step(lam, flux, 'k', lw=1)
plt.title("Mock Spectrum")
plt.show()

## 2. Run MCMC Fit

In [ ]:
res = run_mcmc_fit(
    spec_data=spec_data,
    z_init=z,
    line_names=["H⍺"], # Add ["NII_6549", "NII_6585"] etc.
    grating="PRISM",
    n_steps=2000,
    n_walkers=32,
    verbose=True
)

## 3. Analyze Results

In [ ]:
if res["success"]:
    print("Fit Successful!")
    print(f"Fitted Redshift: {res['z_fit']:.4f}")
    
    # Print Line Parameters
    for name, params in res["lines"].items():
        print(f"\n{name}:")
        print(f"  Flux: {params['flux']:.2f} +/- {params['flux_err']:.2f}")
        print(f"  Sigma(A): {params['sigma_A']:.2f} +/- {params['sigma_A_err']:.2f}")
        
    # Trace Plot
    # Chain shape: (N_samples, N_dim)
    chain = res["chains"]
    plt.figure(figsize=(10,4))
    plt.plot(chain[:, 0], 'k', alpha=0.3)
    plt.title("Trace: Flux H-alpha")
    plt.xlabel("Sample")
    plt.show()
else:
    print("Fit Failed:", res.get("error"))